In [2]:
import pandas as pd
import random
import requests
import json
import os
import time
from flask import Flask, request, jsonify
import threading

# --- Constants and Hyperparameters for Q-learning ---
LEARNING_RATE = 0.1
DISCOUNT_FACTOR = 0.9
EXPLORATION_RATE = 0.2
MOODLE_URL = 'http://localhost:8100/webservice/rest/server.php'
TOKEN = 'bf5eaf838a5f18b8f4b5c2862746caca'  # Local user log plugin token

ACTIONS = [
    'read_new_resource',
    'review_old_resource',
    'attempt_new_quiz',
    'redo_failed_quiz',
    'skip_to_next_module',
    'do_quiz_harder',
    'do_quiz_easier',
    'do_quiz_same'
]

# --- Discretization Bins (storing the lower bound) ---
SCORE_AVG_BINS = [0, 2, 4, 6, 8]
COMPLETE_RATE_BINS = [0.0, 0.3, 0.6]

# --- Global Variables ---
q_table = {}
q_table_lock = threading.Lock()
last_state_action = {}
log_file_path = "/Users/nguyenhuuloc/Documents/MyComputer/moodledata/local_userlog_data/user_log_summary.csv"
user_clusters_df = None
section_ids = []

# --- Helper Functions ---
def load_data():
    """
    Loads necessary data (course hierarchy and user clusters) once.
    """
    global section_ids, user_clusters_df
    try:
        # Load course hierarchy from JSON
        with open('json_course_moodle_hierarchy_clean.json', 'r', encoding='utf-8') as f:
            data = json.load(f)
            # Extract all sectionIdNew from lessons
            for section in data:
                if "lessons" in section:
                    for lesson in section["lessons"]:
                        section_ids.append(lesson["sectionIdNew"])

        # Load user clusters from CSV
        user_clusters_df = pd.read_csv("./synthetic_user_features_clustered.csv")
    except FileNotFoundError as e:
        print(f"❌ Error loading data: {e}. Please ensure files are in the correct directory.")
        exit()

# --- Q-table Management ---
def initialize_q_table(filename='q_table_results.csv'):
    """
    Initializes a new Q-table and saves it to a CSV file.
    """
    initial_q_table = {}
    levels = ['easy', 'medium', 'hard']

    for section_id in section_ids:
        for level in levels:
            for complete_rate in COMPLETE_RATE_BINS:
                for score_bin in SCORE_AVG_BINS:
                    state = (section_id, level, complete_rate, score_bin)
                    initial_q_table[state] = {action: 0.0 for action in ACTIONS}

    save_q_table_to_csv(initial_q_table, filename)
    return initial_q_table

def save_q_table_to_csv(q_table_to_save, filename='q_table_results.csv'):
    """
    Saves the Q-table to a CSV file.
    Note: This function overwrites the entire CSV file with the current in-memory q_table.
    This ensures all updates and existing data are persisted correctly.
    """
    q_table_data = []
    for state, actions_dict in q_table_to_save.items():
        for action, q_value in actions_dict.items():
            row = {
                'sectionid': state[0],
                'level': state[1],
                'complete_rate': state[2],
                'score_bin': state[3],
                'action': action,
                'q_value': q_value
            }
            q_table_data.append(row)
    df = pd.DataFrame(q_table_data)
    df.to_csv(filename, index=False)

def load_q_table_from_csv(filename='q_table_results.csv'):
    """
    Loads the Q-table from a CSV file.
    """
    q_table_loaded = {}
    if not os.path.exists(filename):
        return {}

    try:
        df = pd.read_csv(filename)
        for _, row in df.iterrows():
            state = (row['sectionid'], row['level'], row['complete_rate'], row['score_bin'])
            action = row['action']
            q_value = row['q_value']
            
            if state not in q_table_loaded:
                q_table_loaded[state] = {a: 0.0 for a in ACTIONS}
            q_table_loaded[state][action] = q_value
    except Exception as e:
        print(f"❌ Error loading Q-table from CSV: {e}")
        return {}
    return q_table_loaded

# --- Discretization Functions ---
def discretize_score(score):
    """Discretizes a score from 0-10 into a predefined bin."""
    for b in reversed(SCORE_AVG_BINS):
        if score >= b:
            return b
    return SCORE_AVG_BINS[0]

def discretize_complete_rate(rate):
    """Discretizes a completion rate from 0-1 into a predefined bin."""
    for b in reversed(COMPLETE_RATE_BINS):
        if rate >= b:
            return b
    return COMPLETE_RATE_BINS[0]

# --- User and API Data Fetching ---
def get_user_cluster(user_id):
    """
    Retrieves the user's cluster information from the loaded DataFrame.
    """
    global user_clusters_df
    if user_clusters_df is None:
        return None
    user_row = user_clusters_df[user_clusters_df['userid'] == user_id]
    return user_row.iloc[0]['cluster'] if not user_row.empty else None

def call_moodle_api(function, extra_params):
    """
    Calls the Moodle Web Service API and handles errors.
    """
    params = {
        'wstoken': TOKEN,
        'moodlewsrestformat': 'json',
        'wsfunction': function
    }
    params.update(extra_params)
    try:
        response = requests.post(MOODLE_URL, data=params, timeout=10) # Added a timeout
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error calling Moodle API '{function}': {e}")
        return {}

def safe_get(data, key, default=None):
    """
    Safely retrieves a value from a dictionary or the first element of a list.
    """
    if isinstance(data, dict):
        return data.get(key, default)
    elif isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
        return data[0].get(key, default)
    return default

def get_state_from_moodle(userid, courseid, sectionid, objecttype, objectid):
    """
    Fetches and processes data from Moodle to determine the current state.
    """
    # 1. Average quiz score for a user in a section
    avg_score_data = call_moodle_api('local_userlog_get_user_section_avg_grade', {
        'userid': userid, 'courseid': courseid, 'sectionid': sectionid
    })
    avg_score = safe_get(avg_score_data, 'avg_section_grade', 0)

    # 2. Quiz level from tags
    quiz_level = 'medium'
    if objecttype == "quiz":
        quiz_level_data = call_moodle_api('local_userlog_get_quiz_tags', {'quizid': objectid})
        quiz_level = safe_get(quiz_level_data, 'tag_name', 'medium')

    # 3. Resource completion rate
    total_resources = safe_get(call_moodle_api('local_userlog_get_total_resources_by_section', {
        'sectionid': sectionid, 'objecttypes[0]': 'resource', 'objecttypes[1]': 'hvp',
    }), 'total_resources', 0)
    viewed_resources = safe_get(call_moodle_api('local_userlog_get_viewed_resources_distinct_by_section', {
        'userid': userid, 'courseid': courseid, 'sectionid': sectionid, 'objecttypes[0]': 'resource', 'objecttypes[1]': 'hvp',
    }), 'viewed_resources', 0)
    complete_rate = viewed_resources / total_resources if total_resources > 0 else 0.0

    # 4. Check if latest quiz was passed
    passed_lastest_quiz = safe_get(call_moodle_api('local_userlog_get_latest_quiz_pass_status_by_section', {
        'sectionid': sectionid, 'userid': userid,
    }), 'is_passed', 0) == 1

    # 5. Discretize and return state
    score_bin = discretize_score(avg_score)
    complete_bin = discretize_complete_rate(complete_rate)
    state = (sectionid, quiz_level, complete_bin, score_bin)

    print("\n=== DEBUG get_state_from_moodle ===")
    print(state, passed_lastest_quiz)
    return state, passed_lastest_quiz

# --- Q-learning Core Logic ---
def get_q_value(state, action):
    """Returns the Q-value for a given state-action pair."""
    global q_table
    if state not in q_table:
        q_table[state] = {a: 0.0 for a in ACTIONS}
    return q_table[state][action]

def update_q_table(state, action, reward, next_state):
    """Updates the Q-table using the Q-learning algorithm."""
    global q_table
    current_q = get_q_value(state, action)
    max_future_q = max(q_table[next_state].values()) if next_state in q_table else 0.0
    q_table[state][action] = current_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * max_future_q - current_q)

def get_reward(action, old_score, new_score, old_complete, new_complete,
               cluster=None, quiz_level='medium', passed_hard_quiz=1):
    score_improvement = new_score - old_score
    complete_bonus = (new_complete - old_complete) * 10
    action_bonus = 0
    cluster_bonus = 0

    print("\n=== DEBUG get_reward ===")
    print(f"Action: {action}")
    print(f"old_score={old_score}, new_score={new_score}, score_improvement={score_improvement}")
    print(f"old_complete={old_complete}, new_complete={new_complete}, complete_bonus={complete_bonus}")
    print(f"cluster={cluster}, quiz_level={quiz_level}, passed_hard_quiz={passed_hard_quiz}")

    # --- Logic mới ---
    if action == 'read_new_resource':
        if new_complete >= 1.0:
            action_bonus = -2
        else:
            action_bonus = 1
    elif action == 'review_old_resource':
        if new_score < 5:
            action_bonus = 3
        else:
            action_bonus = 0.5
    elif action == 'redo_failed_quiz':
        if not passed_hard_quiz:
            action_bonus = 4
        else:
            action_bonus = 1
    elif action == 'do_quiz_easier':
        if new_score < 4:
            action_bonus = 5
        else:
            action_bonus = 0.5
    elif action == 'do_quiz_harder':
        if new_score >= 8:
            action_bonus = 4
        else:
            action_bonus = -1

    print(f"action_bonus={action_bonus}")

    # --- Cluster-based personalization ---
    if cluster == 0:  # Yếu
        if action in ['review_old_resource', 'redo_failed_quiz', 'do_quiz_easier']:
            cluster_bonus = 2
    elif cluster == 1:  # Khá/giỏi
        if action in ['do_quiz_harder', 'skip_to_next_module']:
            cluster_bonus = 2

    print(f"cluster_bonus={cluster_bonus}")

    total_reward = score_improvement + complete_bonus + action_bonus + cluster_bonus
    print(f"==> total_reward={total_reward}")
    print("========================\n")

    return total_reward

def suggest_next_action(current_state, userid):
    """
    Selects the next action using an Epsilon-Greedy policy.
    """
    filtered_actions = ACTIONS.copy()
    complete_bin_resource = current_state[2]
    score_bin = current_state[3]
    section_id = current_state[0]
    complete_bin_section = safe_get(call_moodle_api('local_userlog_get_section_completion', {
        'userid': userid, 'sectionid': section_id
    }), 'completion_rate', 0)

    

    print("\n=== DEBUG suggest_next_action ===")
    print(f"Current state: {current_state}")
    print(f"Complete_bin={complete_bin_resource}, Score_bin={score_bin}, Complete_bin_section={complete_bin_section}")

    # --- Lọc action không hợp lý ---
    if complete_bin_resource >= 0.6:
        if 'read_new_resource' in filtered_actions:
            filtered_actions.remove('read_new_resource')
            print("Removed action: read_new_resource (vì complete >= 0.6)")

    if score_bin >= 8:
        if 'do_quiz_easier' in filtered_actions:
            filtered_actions.remove('do_quiz_easier')
            print("Removed action: do_quiz_easier (vì score >= 8)")
            
     # --- Điều kiện thêm skip_to_next_module ---
    if complete_bin_section >= 95 and score_bin >= 8:
        if 'skip_to_next_module' in filtered_actions:
            print("Ưu tiên action: skip_to_next_module (hoàn thành ~xong & điểm khá)")
            return 'skip_to_next_module', get_q_value(current_state, 'skip_to_next_module')

            
    print(f"Filtered actions: {filtered_actions}")

    # --- Exploration or Exploitation ---
    if current_state not in q_table or random.uniform(0, 1) < EXPLORATION_RATE:
        action = random.choice(filtered_actions)
        q_value = get_q_value(current_state, action)
        print(f"[Exploration] Chọn ngẫu nhiên action={action}, q_value={q_value}")
        print("========================\n")
        return action, q_value
    else:
        q_values = {a: q_table[current_state][a] for a in filtered_actions}
        best_action = max(q_values, key=q_values.get)
        print(f"[Exploitation] Chọn action={best_action}, q_value={q_values[best_action]}")
        print("========================\n")
        return best_action, q_values[best_action]

# --- Main Daemon and Flask App ---
def q_learning_daemon():
    """
    Generator that tails the log file like `tail -f`.
    """
    with open(log_file_path, "r") as f:
        f.seek(0, os.SEEK_END)
        while True:
            line = f.readline()
            if not line:
                time.sleep(0.5)
                continue
            yield line.strip()

def process_log_line(line):
    """
    Processes a single log line to update the Q-table.
    """
    global q_table, last_state_action
    
    if not line or line.startswith("userid"):
        return

    try:
        userid, courseid, sectionid, objecttype, objectid, _ = line.split(",", 5)
        userid, courseid, sectionid, objectid = map(int, [userid, courseid, sectionid, objectid])
    except (ValueError, IndexError) as e:
        print(f"❌ Error parsing log line '{line}': {e}")
        return

    print(f"\n📥 New log: user={userid}, section={sectionid}, type={objecttype}")

    current_state, passed_quiz = get_state_from_moodle(
        userid, courseid, sectionid, objecttype, objectid
    )
    
    with q_table_lock:
        if userid in last_state_action:
            prev_state, prev_action = last_state_action[userid]

            if prev_state[0] == current_state[0]:
                reward = get_reward(
                    action=prev_action,
                    old_score=prev_state[3],
                    new_score=current_state[3],
                    old_complete=prev_state[2],
                    new_complete=current_state[2],
                    cluster=get_user_cluster(userid),
                    quiz_level=prev_state[1],
                    passed_hard_quiz=passed_quiz,
                )
                print(f"🏆 Reward for user {userid}: {reward:.2f}")
                update_q_table(prev_state, prev_action, reward, current_state)
                save_q_table_to_csv(q_table)
            else:
                print("❗ Log from a new section, skipping Q-table update.")

        action, _ = suggest_next_action(current_state, userid=userid)
        last_state_action[userid] = (current_state, action)

app = Flask(__name__)

@app.route('/api/suggest-action', methods=['POST'])
def suggest_action_api():
    """
    API endpoint to get the next suggested action for a user.
    """
    data = request.json
    if not data:
        return jsonify({"error": "No data provided"}), 400
    
    try:
        userid = int(data.get('userid'))
        courseid = int(data.get('courseid'))
        sectionid = int(data.get('sectionid'))
        objecttype = data.get('type')
        objectid = int(data.get('objectid'))
    except (TypeError, ValueError) as e:
        return jsonify({"error": f"Invalid data types: {e}"}), 400

    print(f"\n💡 API request for user={userid}, section={sectionid}")
    
    current_state, _ = get_state_from_moodle(userid, courseid, sectionid, objecttype, objectid)

    with q_table_lock:
        action, q_value = suggest_next_action(current_state, userid=userid)
        
    print(f"✨ Suggested action for user {userid}: {action} (Q={q_value:.2f})")

    return jsonify({
        "user_id": userid,
        "suggested_action": action,
        "q_value": q_value,
        "current_state": {
            "section_id": current_state[0],
            "quiz_level": current_state[1],
            "complete_rate_bin": current_state[2],
            "score_bin": current_state[3]
        }
    })
    
@app.route('/api/suggest-latest-action', methods=['GET'])
def suggest_latest_action_api():
    """
    (Endpoint mới) API endpoint không có tham số để lấy gợi ý dựa trên hành động gần nhất của bất kỳ người dùng nào.
    """
    with q_table_lock:
        if not last_state_action:
            return jsonify({"message": "No recent user activity to suggest an action from."}), 404
            
        # Lấy trạng thái từ hành động gần nhất của bất kỳ người dùng nào
        # Chỉ lấy 1 user và state gần nhất
        last_userid, (current_state, last_action) = next(iter(last_state_action.items()))
        
        # Đưa ra một gợi ý mới dựa trên trạng thái đó
        action, q_value = suggest_next_action(current_state, last_userid)
        
    print(f"✨ Suggested a new action from latest user state: {action} (Q={q_value:.2f})")
    
    return jsonify({
        "message": "Suggestion based on the latest logged user activity.",
        "suggested_action": action,
        "q_value": q_value,
        "source_state": {
            "section_id": current_state[0],
            "quiz_level": current_state[1],
            "complete_rate_bin": current_state[2],
            "score_bin": current_state[3]
        }
    })


@app.route('/api/full-learning-path', methods=['POST'])
def full_learning_path():
    """
    API trả về toàn bộ lộ trình học của user trong một course.
    Mỗi section sẽ có nhiều action được gợi ý cho đến khi complete_rate = 1.0 và score_bin >= 8
    """
    data = request.json
    try:
        userid = int(data.get('userid'))
        courseid = int(data.get('courseid'))
    except (TypeError, ValueError):
        return jsonify({"error": "Invalid userid or courseid"}), 400

    learning_path = []

    # Lặp qua tất cả section trong course
    for section in section_ids:  # Nếu có nhiều course, có thể lọc section theo courseid
        section_path = []
        
        # Lấy state ban đầu của user ở section này
        current_state, passed_quiz = get_state_from_moodle(userid, courseid, section, "quiz", 0)

        # Giả lập hành động nhiều bước trong section
        steps = 0
        while steps < 20:  # giới hạn số action để tránh vòng lặp vô hạn
            action, q_value = suggest_next_action(current_state, userid)
            section_path.append({
                "state": {
                    "score_bin": current_state[3],
                    "complete_bin": current_state[2],
                    "quiz_level": current_state[1]
                },
                "suggested_action": action,
                "q_value": q_value
            })

            # Giả lập update state dựa trên action
            # Đây là mô phỏng đơn giản: tăng complete hoặc score dựa trên loại action
            score_bin = current_state[3]
            complete_bin = current_state[2]

            if action in ['read_new_resource', 'review_old_resource']:
                complete_bin = min(1.0, complete_bin + 0.3)
            if action in ['attempt_new_quiz', 'redo_failed_quiz', 'do_quiz_harder', 'do_quiz_easier', 'do_quiz_same']:
                score_bin = min(10, score_bin + 2)

            new_state = (current_state[0], current_state[1], discretize_complete_rate(complete_bin), discretize_score(score_bin))
            
            if new_state == current_state or (new_state[2] == 0.6 and new_state[3] >= 8):
                # Section đã gần hoàn thành, dừng lại
                break

            current_state = new_state
            steps += 1

        learning_path.append({
            "section_id": section,
            "section_path": section_path
        })

    return jsonify({
        "user_id": userid,
        "course_id": courseid,
        "full_learning_path": learning_path
    })


if __name__ == '__main__':
    load_data()
    q_table = load_q_table_from_csv()
    if not q_table:
        print("⚠️ Q-table not found, initializing a new one.")
        q_table = initialize_q_table()

    # Start the Q-learning daemon in a separate thread
    daemon_thread = threading.Thread(target=lambda: (
        print("🚀 Q-learning daemon started. Monitoring log file..."),
        [process_log_line(line) for line in q_learning_daemon()]
    ))
    daemon_thread.daemon = True
    daemon_thread.start()

    # Run the Flask app
    app.run(debug=True, port=8088, use_reloader=False)


🚀 Q-learning daemon started. Monitoring log file...
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8088
Press CTRL+C to quit



=== DEBUG get_state_from_moodle ===
(38, 'medium', 0.6, 8) False

=== DEBUG suggest_next_action ===
Current state: (38, 'medium', 0.6, 8)
Complete_bin=0.6, Score_bin=8, Complete_bin_section=100
Removed action: read_new_resource (vì complete >= 0.6)
Removed action: do_quiz_easier (vì score >= 8)
Ưu tiên action: skip_to_next_module (hoàn thành ~xong & điểm khá)

=== DEBUG get_state_from_moodle ===
(39, 'medium', 0.6, 8) True

=== DEBUG suggest_next_action ===
Current state: (39, 'medium', 0.6, 8)
Complete_bin=0.6, Score_bin=8, Complete_bin_section=60
Removed action: read_new_resource (vì complete >= 0.6)
Removed action: do_quiz_easier (vì score >= 8)
Filtered actions: ['review_old_resource', 'attempt_new_quiz', 'redo_failed_quiz', 'skip_to_next_module', 'do_quiz_harder', 'do_quiz_same']
[Exploitation] Chọn action=review_old_resource, q_value=7.845636103723425


=== DEBUG get_state_from_moodle ===
(40, 'medium', 0.6, 8) True

=== DEBUG suggest_next_action ===
Current state: (40, 'medium'

127.0.0.1 - - [25/Aug/2025 08:53:18] "POST /api/full-learning-path HTTP/1.1" 200 -



=== DEBUG get_state_from_moodle ===
(43, 'medium', 0.6, 8) True

=== DEBUG suggest_next_action ===
Current state: (43, 'medium', 0.6, 8)
Complete_bin=0.6, Score_bin=8, Complete_bin_section=0
Removed action: read_new_resource (vì complete >= 0.6)
Removed action: do_quiz_easier (vì score >= 8)
Filtered actions: ['review_old_resource', 'attempt_new_quiz', 'redo_failed_quiz', 'skip_to_next_module', 'do_quiz_harder', 'do_quiz_same']
[Exploration] Chọn ngẫu nhiên action=do_quiz_same, q_value=0.0




📥 New log: user=4, section=40, type=hvp

=== DEBUG get_state_from_moodle ===
(40, 'medium', 0.6, 8) True

=== DEBUG suggest_next_action ===
Current state: (40, 'medium', 0.6, 8)
Complete_bin=0.6, Score_bin=8, Complete_bin_section=0
Removed action: read_new_resource (vì complete >= 0.6)
Removed action: do_quiz_easier (vì score >= 8)
Filtered actions: ['review_old_resource', 'attempt_new_quiz', 'redo_failed_quiz', 'skip_to_next_module', 'do_quiz_harder', 'do_quiz_same']
[Exploitation] Chọn action=review_old_resource, q_value=0.0



Exception in thread Thread-7 (<lambda>):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/nguyenhuuloc/Library/Python/3.12/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/g4/mfpnrdbj2zj_69p2jw0y5nnc0000gn/T/ipykernel_19534/4127054911.py", line 551, in <lambda>
  File "/var/folders/g4/mfpnrdbj2zj_69p2jw0y5nnc0000gn/T/ipykernel_19534/4127054911.py", line 374, in process_log_line
  File "/var/folders/g4/mfpnrdbj2zj_69p2jw0y5nnc0000gn/T/ipykernel_19534/4127054911.py", line 209, in get_state_from_moodle
  File "/var/folders/g4/mfpnrdbj2zj_69p2jw0y5nnc0000gn/T/ipykernel_19534/4127054911.py", line 128, in discretize_score
TypeError


📥 New log: user=2, section=40, type=hvp
